# Compute the spectrum of Fe X

In [1]:
using JAC

Multiply charged ions have attracted continuous interest due to recent observations by the Solar and heliospheric Observatory (SOHO), the Far Ultraviolet Spectroscopic Explorer (FUSE) and several other astrophysical missions. In the extreme ultra-violet (EUV) range of spectra, for instance, many of the spectroscopically observed lines belong to the iron-group elements owing to their high abundance in different objects. Until the present, however, several of these lines remained unclassified because reliable theoretical predictions are often not available that consistently include relativistic and correlation effects. 

To demonstrate the use of the JAC toolbox, we here consider the level structure of Fe$^{9+}$ ions which, in spectroscopic notation, give rise to the spectrum Fe X. Several medium-to-large scale computations have been carried out to calculate the excitation energies and transition probabilities of (and among) the levels from the [Ne] $3s^2 3p^5$ -> $3s^2 3p^4 3d + 3s 3p^6$ configuration. These three configurations comprise in total 31 fine-structure levels with total angular momenta $J = 1/2, ..., 9/2$ (cf. Dong et al., 1999). Although the $3s^2 3p^4 3d + 3s 3p^6$ levels mainly decay by electric-dipole (E1) transitions, also M1, E2 and M2 multipole transitions have been considered in the literature. Below, we show how this *complete* Fe X spectrum can be calculated approximately by just a single call within the JAC toolbox.



However, before we shall perform this computation, let us define a standard grid for the computations:

In [2]:
grid = Radial.Grid(true)
setDefaults("standard grid", grid)

(Re-) Define the standard grid with 390 grid points.


Moreover, we can show here how to select proper **physical units**, in which all the energies, rates, etc. are handled at input and output time. To see the present `settings` of this units, we can simply call:

In [3]:
Basics.display("settings")

Current settings of the JAC module:  
-----------------------------------  

  + Framework:                              relativistic
  + Energy unit:                            eV
  + Rate and transition probability unit:   1/s
  + Cross section unit:                     barn
  + Time unit:                              sec

  + A standard grid has been defined; cf. Defaults.getDefaults()



Of course, we could easily change any of these units by a proper call to `setDefaults()`, but we shall leave the present (standard) choice for the given computations. Call `setDefaults()` to change these settings:


In [4]:
? setDefaults

search: setDefaults



`Defaults.setDefaults()`       ... (re-) defines some 'standard' settings which are common to all the computations with the JAC module, and which can          be 'overwritten' by the user. –- An improper setting of some variable may lead to an error message, if recognized         immediately. The following defaults apply if not specified otherwise by the user: the framework is 'relativistic',          energies are given in eV and cross sections in barn. Note that, internally, atomic units are used throughout for          all the computations within the program. nothing is returned if not indicated otherwise.

  * `("framework: relativistic")`  or  `("framework: non-relativistic")`      ... to define a relativistic or non-relativistic framework for all subsequent computations.
  * `("method: continuum, spherical Bessel")`  or  `("method: continuum, pure sine")`  or   `("method: continuum, asymptotic Coulomb")`  or  `("method: continuum, nonrelativistic Coulomb")`  or   `("method: continuum, Galerkin")`     ... to define a a method for the generation of the continuum orbitals as (pure) spherical Bessel, pure sine,       asymptotic Coulomb, nonrelativistic Coulomb orbital or by means of the B-spline-Galerkin method.
  * `("method: normalization, pure sine")`  or  `("method: normalization, pure Coulomb")`  or  `("method: normalization, Ong-Russek")`      ... to define a method for the normalization of the continuum orbitals as asymptotically (pure) sine or Coulomb        functions, or following the procedure by Ong & Russek (1978).
  * `("QED model: Petersburg")`  or  `("QED model: Sydney")`      ... to define a model for the computation of the QED corrections following the work by Shabaev et al. (2011; Petersburg)        or Flambaum and Ginges (2004; Syney).
  * `("unit: energy", "eV")`  or  `("unit: energy", "Kayser")`  or  `("unit: energy", "Hartree")`  or   `("unit: energy", "Hz")`  or  `("unit: energy", "Hz")`     ... to (pre-) define the energy units for all further printouts and communications with the JAC module.
  * `("unit: cross section", "a.u.")`  or  `("unit: cross section", "barn")`  or  `("unit: cross section", "Mbarn")`     ... to (pre-) define the unit for the printout of cross sections.
  * `("unit: rate", "a.u.")`  or  `("unit: rate", "1/s")`  ... to (pre-) define the unit for the printout of rates.
  * `("unit: time", "a.u.")`  or  `("unit: time", "sec")`  or  `("unit: time", "fs")`  or  `("unit: time", "as")`     ... to (pre-) define the unit for the printout and communications of times with the JAC module.

---

  * `("relativistic subshell list", subshells::Array{Subshell,1}; printout::Bool=true)`     ... to (pre-) define internally the standard relativistic subshell list on which the standard order of orbitals is based.

---

  * `("standard grid", grid::Radial.Grid; printout::Bool=true)`     ... to (pre-) define internally the standard radial grid which is used to represent most orbitals.

---

  * `("QED: damped-hydrogenic", Znuc::Float64, wa::Array{Float64,1})`     ... to (re-) define the lambda-C damped overlap integrals of the lowest kappa-orbitals        [ wa*1s*1/2, wa*2p*1/2, wa*2p*3/2, wa*3d*3/2, wa*3d*5/2 ] for the (new) nuclear charge Znuc;        nothing is returned.



Apparently, we can call `JAC.define()` also in order to open some ASCII (.txt) file into which all the final (important) results are printed for later reference and inspection.


In [5]:
setDefaults("print summary: open", "09-output-radiative.txt")


Next, we shall now describe the computation to be carried out. Apart from a title line (String) and nuclear model (cf. tutorial 03-setting-the-nucleus.ipynb), we here need to specify the initial- and final-state configurations that are to be included into the computation. This is done by defining an instance of an:

```julia
comp = Atomic.Computation(Atomic.Computation(), name="Energies and Einstein coefficients for the spectrum Fe X", 
              grid=Radial.Grid(true), nuclearModel=Nuclear.Model(26.);              
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],              
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")],               
              process = Radiative,  processSettings = PhotoEmission.Settings(...) ); 
```

In principle, any number of configurations can be specified as long as they are listed explicitly. Here, we have selected also the identifier `Radiative` to consider the computation of transition probabilities between the levels of the three configurations given above. All further details about any computation can be specified by means of some proper `settings` that are specific to each particular process, here `PhotoEmission.Settings()`. To better understand to which extent we can affect and control these computations, let us look at the definition of this data struct:


In [6]:
? PhotoEmission.Settings

`struct  PhotoEmission.Settings`  ... defines a type for the details and parameters of computing radiative lines.

```
+ multipoles              ::Array{EmMultipoles}     ... Specifies the (radiat. field) multipoles to be included.
+ gauges                  ::Array{UseGauge}         ... Gauges to be included into the computations.
+ calcAnisotropy          ::Bool                    ... True, if the anisotropy (structure) functions are to be 
                                                        calculated and false otherwise 
+ printBefore             ::Bool                    ... True, if all energies and lines are printed before comput.
+ selectLines             ::Bool                    ... True, if lines are selected individually for the computat.
+ selectedLines           ::Array{Tuple{Int64,Int64},1}  ... List of lines as tupels (inital-level, final-level).
+ photonEnergyShift       ::Float64                 ... An overall energy shift for all photon energies.
+ mimimumPhotonEnergy     ::Float64                 ... minimum transition energy for which (photon) transitions 
                                                        are included into the computation.
+ maximumPhotonEnergy     ::Float64                 ... maximum transition energy for which (photon) transitions 
                                                        are included.
```

---

`PhotoEmission.Settings()`  ... constructor for the default values of radiative line computations

---

`PhotoEmission.Settings(set::PhotoEmission.Settings;`

```
    multipoles::=..,        gauges=..,          calcAnisotropy=..,          printBefore=..,
    selectLines=..,         selectedLines=..,   photonEnergyShift=..,       mimimumPhotonEnergy=.., 
    maximumPhotonEnergy=..) 
                
... constructor for modifying the given PhotoEmission.Settings by 'overwriting' the previously selected parameters.
```


Of course, we can always apply the default values for any particular compuation which, for present case, are given by:

In [7]:
defaultsSettings = PhotoEmission.Settings()

multipoles:             EmMultipole[E1]  
gauges:                 UseGauge[UseCoulomb]  
calcAnisotropy:         false  
printBefore:            false  
selectLines:            false  
selectedLines:          Tuple{Int64,Int64}[]  
photonEnergyShift:      0.0  
mimimumPhotonEnergy:    0.0  
maximumPhotonEnergy:    10000.0  



Below, we wish to consider also other multipole transitions, namely E1, M1, E2, M2, and like perform all calculations within both, the Coulomb and Babushkin gauges. In addition, we shall **NOT** compute the (angular) anisotropy parameters nor do we wish to select individual transitions right from the beginning. This latter option might be interesting if large sets of configurations are to be considered. Moreover, we do not neither specify any (overall) shift nor restrict the photon energies, two features that are of particular interest for large data sets


We these remarks in mind, we now specify the atomic computation in the form:


In [8]:
photoSettings = PhotoEmission.Settings(defaultsSettings, multipoles=[E1, M1, E2, M2], gauges=[UseCoulomb, UseBabushkin],
                                       printBefore=true)

multipoles:             EmMultipole[E1, M1, E2, M2]  
gauges:                 UseGauge[UseCoulomb, UseBabushkin]  
calcAnisotropy:         false  
printBefore:            true  
selectLines:            false  
selectedLines:          Tuple{Int64,Int64}[]  
photonEnergyShift:      0.0  
mimimumPhotonEnergy:    0.0  
maximumPhotonEnergy:    10000.0  


In [13]:
comp = Atomic.Computation(Atomic.Computation(), name="Energies and Einstein coefficients for the spectrum Fe X",  
              grid=grid, nuclearModel=Nuclear.Model(26.);
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")], 
              process = Radiative(), processSettings = photoSettings ); 


All what is needed now is to **perform** this computation by the call:


In [14]:
perform(comp)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^1 3p_1/2^2 3p_3/2^4 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^0 3p_3/2^4 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^0 3p_3/2^4 3d_3/2^1 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_3/2^1 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_3/2^1 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.75657341


... and which shows all the intermediate output of the computations. First, all the initial-state levels are calculated *self-consistently* due to the default values for such SCF and configuration-interaction computations, and similarly for all final-state levels. These computations include the definition of the CSF basis, the generation of *start* orbitals as well as the computation of the angular coefficients, the SCF interation and up to the set-up and diagonalization of the corresponding Hamiltonian matrix. Obviously, this gives rise to a full representation of all atomic levels of interest.

These levels are then utilized to compute the (multipole) transitions amplitudes and rates. All final results are also tabulated in a neat format. However, since many of the internal tables of the JAC toolbox are *wider* than the standard widths of these notebooks, we shall first **close** our .txt summary file:


In [11]:
setDefaults("print summary: close", "")


This summary file **09-output-radiative.txt** can now be found in the current directory. As seen from these tables, the transition energies agree within 3..4 digits with a similar computation by other atomic structure codes, such as GRASP. Apart from the level splitting among the low-lying $3s 3p^6$, $3s^2 3p^4 3d$  $J=1/2$ levels, further comparision can be made also with the evaluated data from the NIST Spectroscopic Database, for instance, for the excitation energies from the $3s^2 3p^5$  $J = 3/2$ ground level.